In [1]:
packages = [
    "torch==2.0.1",
    "torch-geometric==2.3.1",
    "numpy==1.23.0",
    "pandas==1.5.3"
]
import warnings
warnings.filterwarnings("ignore")
# Instalación de cada paquete uno por uno
for package in packages:
    !pip install {package}

ERROR: Could not find a version that satisfies the requirement torch==2.0.1 (from versions: 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1)
ERROR: No matching distribution found for torch==2.0.1


     ---------------------------------------- 0.0/661.6 kB ? eta -:--:--
     -------------------------------------- 661.6/661.6 kB 6.3 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ------ --------------------------------- 1.8/11.1 MB 12.6 MB/s eta 0:00:01
   ---------------------------------- ----- 9.7/11.1 MB 31.8 MB/s eta 0:00:01
   ---------------------------------------- 11.1/11.1 MB 27.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/44.5 MB ? eta -:--:--
   -------- ------------------------------- 9.4/44.5 MB 45.2 MB/s eta 0:00:01
   ----------------- ---------------------- 19.9/44.5 MB 46.6 MB

  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [33 lines of output]
      Traceback (most recent call last):
        File "C:\Users\USUARIO\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
          main()
        File "C:\Users\USUARIO\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "C:\Users\USUARIO\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 112, in get_requires_for_build_wheel
          backend = _build_backend()
                    ^^^^^^^^^^^^^^^^
        File "C:\Users\USUARIO\AppData\Local\Programs\Python\Python31

     ---------------------------------------- 0.0/5.2 MB ? eta -:--:--
     -------------------- ------------------- 2.6/5.2 MB 21.6 MB/s eta 0:00:01
     ---------------------------------------- 5.2/5.2 MB 26.3 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: still running...
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build pandas


  error: subprocess-exited-with-error
  
  × Building wheel for pandas (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [2514 lines of output]
      <string>:19: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
      running bdist_wheel
      running build
      running build_py
      creating build\lib.win-amd64-cpython-312\pandas
      copying pandas\conftest.py -> build\lib.win-amd64-cpython-312\pandas
      copying pandas\testing.py -> build\lib.win-amd64-cpython-312\pandas
      copying pandas\_typing.py -> build\lib.win-amd64-cpython-312\pandas
      copying pandas\_version.py -> build\lib.win-amd64-cpython-312\pandas
      copying pandas\__init__.py -> build\lib.win-amd64-cpython-312\pandas
      creating build\lib.win-amd64-cpython-312\pandas\api
      copying pandas\api\__init__.py -> build\lib.win-amd64-cpython-312\pandas\api
      creating build\lib.win-amd64-cpython-312\pandas\arrays
    

In [2]:
!pip install torch
!pip install --upgrade numpy pandas
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch_geometric.data import Data, DataLoader
from torch.distributions import Categorical
import numpy as np
import pandas as pd

   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   -------------- ------------------------- 4.7/12.6 MB 25.9 MB/s eta 0:00:01
   ---------------------------------------- 12.6/12.6 MB 32.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.3
    Uninstalling numpy-2.1.3:
      Successfully uninstalled numpy-2.1.3


In [3]:
#set input dates
date = "_2024-12-09"
truck = "VAN"
mpg = 6.5

In [52]:
# Specify the file name
file_name1 = "rate_q2_west_"+truck+date+".csv"
file_name2 = "duration_west.csv"
file_name3 = "prob_west_"+truck+date+".csv"
file_name4 = "load_av_west_"+truck+date+".csv"
file_name5 = "distance_west.csv"
file_name6 = "diesel_west"+date+".csv"
file_name7 = "labels_west.csv"
# Read the Excel file into a DataFrame
rate_matrix = pd.read_csv(file_name1,header= None)
time_matrix = pd.read_csv(file_name2,header = None)
markov_matrix = pd.read_csv(file_name3,header=None)
loads_matrix = pd.read_csv(file_name4,header=None)
distance_matrix = pd.read_csv(file_name5,header=None)
diesel_matrix = pd.read_csv(file_name6,header=None)
hub_labels = pd.read_csv(file_name7,header=None)
initial_node = 9
time_limit = 40

In [53]:
revenue_matrix = rate_matrix * distance_matrix
revenue_matrix[loads_matrix <= 1] = 0
diesel_matrix = diesel_matrix / mpg
var_cost_matrix = 1.2 * distance_matrix
cost_matrix = distance_matrix * diesel_matrix
cost_matrix = cost_matrix + 163
cost_matrix = cost_matrix + var_cost_matrix
reward_matrix = revenue_matrix - cost_matrix
time_matrix *= 0.9

In [54]:
class GraphEnvironment:
    def __init__(self, reward_matrix, time_matrix, markov_matrix, initial_node, time_limit, loads_matrix):
        self.reward_matrix = reward_matrix
        self.time_matrix = time_matrix
        self.markov_matrix = markov_matrix
        self.initial_node = initial_node
        self.time_limit = time_limit
        self.loads_matrix = loads_matrix
        self.num_nodes = reward_matrix.shape[0]
        self.edge_index = self.generate_edge_index()
        
    def generate_edge_index(self):
        edges = []
        for i in range(self.num_nodes):
            for j in range(self.num_nodes):
                if i != j:
                    edges.append((i, j))
        edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
        edge_index = edge_index[1]
        return edge_index

    def reset(self):
        self.current_node = self.initial_node
        self.remaining_time = self.time_limit
        self.visited_nodes = [self.initial_node]
        self.visited_edges = []
        self.total_reward = 0
        return self.get_state()

    def get_state(self):
        return torch.tensor([self.current_node, self.remaining_time], dtype=torch.float32)

    def is_terminal(self):
        # Helper function for terminal conditions
        return (self.remaining_time <= -self.time_limit * 0.3) and (self.current_node == self.initial_node)

    def step(self, action):
        next_node = action.item()
        edge = (self.current_node, next_node)

        if edge in self.visited_edges or self.current_node == next_node:
            # Handle invalid moves (e.g., repeat or self-loop)
            return self.get_state(), 0, False, {}, self.visited_nodes

        self.visited_edges.append(edge)
        reward = self.reward_matrix.iloc[self.current_node, next_node]
        time_consumption = self.time_matrix.iloc[self.current_node, next_node]
        self.total_reward += reward
        self.remaining_time -= time_consumption
        self.current_node = next_node
        self.visited_nodes.append(next_node)

        if self.is_terminal():
            done = True
            # Optionally include final rewards or reset if necessary
            return self.get_state(), reward, done, {}, self.visited_nodes
            
        # Check if time to return to start
        if -self.time_limit * 0.15 < self.remaining_time <= 0:
            edge = (self.current_node, self.initial_node)
            reward = self.reward_matrix.iloc[self.current_node, self.initial_node]
            #loads = self.loads_matrix.iloc[self.current_node, self.initial_node]
            time_consumption = self.time_matrix.iloc[self.current_node, self.initial_node]
            self.total_reward += reward
            self.remaining_time -= time_consumption
            self.current_node = self.initial_node
            self.visited_nodes.append(self.initial_node)
            done = (self.current_node == self.initial_node)
            return self.get_state(), reward, done, {},self.visited_nodes
        elif (len(self.visited_nodes) == self.num_nodes) or (self.remaining_time <= -self.time_limit*0.15):
            self.current_node = self.initial_node
            self.remaining_time = self.time_limit
            self.visited_nodes = [self.initial_node]
            self.visited_edges = []
            self.total_reward = 0
            return self.get_state(), 0, False, {},self.visited_nodes

        # Not terminal and valid move
        return self.get_state(), reward, False, {}, self.visited_nodes

In [55]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.attention_weights = nn.Linear(hidden_size, 1, bias=False)

    def forward(self, x):
        # Compute attention scores and normalize
        scores = self.attention_weights(x)
        weights = F.softmax(scores, dim=1)
        # Apply attention weights to the input
        attended_output = torch.bmm(weights.transpose(1, 2), x).squeeze(1)
        return attended_output

class AttentionDQN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(AttentionDQN, self).__init__()
        
        # Initial linear layers
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        
        # Attention layers
        self.attention_layer = nn.Linear(hidden_size, hidden_size)
        
        # Final linear layers after attention
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, output_size)

    def forward(self, state):
        # Standard forward pass through initial layers
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        
        # Attention mechanism
        # Expand dimensions to create compatibility for bmm
        x_expanded = x.unsqueeze(1)  # Shape: [batch_size, 1, hidden_size]
        
        attention_scores = F.softmax(self.attention_layer(x), dim=-1)  # Shape: [batch_size, hidden_size]
        attention_scores = attention_scores.unsqueeze(2)  # Shape: [batch_size, hidden_size, 1]
        
        # Apply attention to the input
        attended_output = torch.bmm(x_expanded, attention_scores).squeeze(2)  # Shape: [batch_size, hidden_size]
        
        # Continue forward pass after attention
        x = F.relu(self.fc3(attended_output))
        x = self.fc4(x)
        return x
    
class SimpleDQN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleDQN, self).__init__()
        
        # Input layer: maps input of size 2 to the hidden size
        self.fc1 = nn.Linear(input_size, hidden_size)
        
        # Attention layer: self-attention based mechanism to align with hidden size
        self.attention_layer = nn.Linear(hidden_size, hidden_size)  # Matches the hidden dimension
        self.attention_score = nn.Linear(hidden_size, 1)
        
        # Additional fully connected layers
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)  # Maps to the output size for Q-values
        
    def forward(self, x):
        # First layer transformation
        x = F.relu(self.fc1(x))  # Project input to hidden size
        
        # Attention mechanism
        attention_scores = F.softmax(self.attention_score(x), dim=1)
        attended_output = attention_scores * x  # Element-wise attention application
        
        # Process through the remaining layers
        x = F.relu(self.fc2(attended_output))
        return self.fc3(x)

In [56]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
env = GraphEnvironment(reward_matrix, time_matrix, markov_matrix, initial_node, time_limit,loads_matrix)
# Define hyperparameters
learning_rate = 0.001
gamma = 0.5
epsilon_start = 1.0
epsilon_decay = 0.995
epsilon_min = 0.01
num_episodes = 700
hidden_size = env.num_nodes * (env.num_nodes - 1)  # Set according to number of arcs

# Initialize the model with attention mechanism
input_size = 2  # state dimensions: current node, remaining time
output_size = len(env.edge_index)  # number of possible actions (edges)
model = SimpleDQN(input_size, hidden_size, output_size)  # Using the new Attention DQN
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = nn.MSELoss()

# Set epsilon
epsilon = epsilon_start

# Action selection with epsilon-greedy policy
def select_action(state, epsilon):
    if np.random.rand() < epsilon:
        a_type = 1
        options = env.markov_matrix.iloc[int(state[0].item())]
        probabilities = options / options.sum()
        action = torch.tensor(int(np.random.choice(options.index, p=probabilities.values) + ((state[0].item()) * (env.num_nodes - 1))))
        return action, a_type
    else:
        with torch.no_grad():
            q_values = model(state.unsqueeze(0))
            a_type = 2
            return q_values.argmax(dim=-1).unsqueeze(0), a_type


In [57]:
# Training loop
origins= [9,11,14,41,28,0]
for i in origins:  # Iterate through each possible starting node
    stop_counter = 0
    best_reward = 0
    last_reward = 0
    epsilon = epsilon_start
    episode = 0
    patience_counter = 150

    while stop_counter < 10 and episode < num_episodes and epsilon > epsilon_min and patience_counter > 0:
        initial_node = i
        env = GraphEnvironment(reward_matrix, time_matrix, markov_matrix, initial_node, time_limit, loads_matrix)
        state = env.reset()
        done = False
        total_reward = 0

        # Adjust epsilon for each episode
        epsilon = max(epsilon * epsilon_decay, epsilon_min)

        while not done:
            # Choose an action using epsilon-greedy policy
            action, _ = select_action(state, epsilon)
            new_node = env.edge_index[action.item()]

            # Execute the action in the environment
            next_state, reward, done, _, visited_nodes = env.step(new_node)
            total_reward += reward

            # Calculate the target
            with torch.no_grad():
                # Reshape `next_state` to fit the model's expected input (batch size, feature size)
                next_state_expanded = next_state.unsqueeze(0)  # Adding batch dimension if needed

                # Compute target
                if done:
                    target = torch.tensor(reward, dtype=torch.float32)
                else:
                    next_q_values = model(next_state_expanded)  # Predict Q-values for the next state
                    max_next_q_value = next_q_values.max()  # Get the max Q-value for the best action in the next state
                    target = torch.tensor(reward, dtype=torch.float32) + gamma * max_next_q_value


            # Update model based on state-action pair and loss
            state_expanded = state.unsqueeze(0)
            q_values = model(state_expanded).squeeze(0)
            loss = loss_fn(q_values[action.item()].float(), target.float())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Update state for the next step
            state = next_state

        # Check for reward improvement and update stopping criteria
        if total_reward > best_reward:
            patience_counter = 100 if total_reward > best_reward * 1.1 else patience_counter - 1
            best_reward = total_reward
        else:
            patience_counter -= 1

        stop_counter = stop_counter + 1 if last_reward == total_reward else 0
        last_reward = total_reward

        # Print progress
        print(f"Episode {episode + 1}, Reward: {int(total_reward)}, Start Node: {i}, Visited Nodes: {visited_nodes}","Counters: PC :",patience_counter," SC :",stop_counter)
        episode += 1

Episode 1, Reward: -527, Start Node: 9, Visited Nodes: [9, 18, 26, 42, 9] Counters: PC : 149  SC : 0
Episode 2, Reward: 2825, Start Node: 9, Visited Nodes: [9, 19, 39, 22, 9] Counters: PC : 100  SC : 0
Episode 3, Reward: 483, Start Node: 9, Visited Nodes: [9, 17, 4, 9] Counters: PC : 99  SC : 0
Episode 4, Reward: -183, Start Node: 9, Visited Nodes: [9, 42, 47, 9] Counters: PC : 98  SC : 0
Episode 5, Reward: -565, Start Node: 9, Visited Nodes: [9, 13, 38, 9] Counters: PC : 97  SC : 0
Episode 6, Reward: 2792, Start Node: 9, Visited Nodes: [9, 42, 47, 9] Counters: PC : 96  SC : 0
Episode 7, Reward: 498, Start Node: 9, Visited Nodes: [9, 41, 1, 9] Counters: PC : 95  SC : 0
Episode 8, Reward: 459, Start Node: 9, Visited Nodes: [9, 19, 15, 9] Counters: PC : 94  SC : 0
Episode 9, Reward: 64, Start Node: 9, Visited Nodes: [9, 15, 33, 41, 9] Counters: PC : 93  SC : 0
Episode 10, Reward: -484, Start Node: 9, Visited Nodes: [9, 29, 32, 36, 9] Counters: PC : 92  SC : 0
Episode 11, Reward: -490, St

Display Results and Route Solutions

In [10]:
def calculate_days(route, time_matrix):
  total_time = 0
  for i in range(0,len(route)-1):
    lane_time = time_matrix.iloc[route[i],route[i+1]]
    total_time = total_time + lane_time + 2
  days = int(np.floor(total_time / 11))
  return days

In [11]:
def visualize_route (route,hub_labels):
  visualized_route = []
  for i in range(0,len(route)):
    name = hub_labels.at[route[i], 0]
    visualized_route.append(name)
  return visualized_route

In [12]:
def save_best(route, reward, best_profits, most_profit_routes):
    if reward > best_profits.min():
        # Find the index of the lowest value in best_profits
        min_index = best_profits.idxmin()
        
        # Replace the lowest value with the new reward
        best_profits[min_index] = reward
        
        # Replace the corresponding route in most_profit_routes
        most_profit_routes[min_index] = route
        
    return best_profits, most_profit_routes

In [58]:
type1=0
type2=0
type3=0
type4=0
all_routes =[]
# Inference loop
for i in origins:
    for j in range(0,5):
      taken_actions = []
      # Reset environment to get initial state
      # Initialize environment
      initial_node=i
      env = GraphEnvironment(reward_matrix, time_matrix, markov_matrix, initial_node, time_limit,loads_matrix)
      state = env.reset()
      # Episode loop
      done = False
      q_values = model(state.unsqueeze(0))
      while not done:
          # Select action using the trained model
          with torch.no_grad():
              q_values = model(state.unsqueeze(0))
              # Apply action mask to zero out probabilities of previously taken actions
              action_mask = torch.zeros_like(q_values)
              # Set probabilities of previously taken actions to -infinity
              action_mask[0,taken_actions] = float('-1000000')
              masked_q_values = q_values + action_mask
              probabilities = F.softmax(masked_q_values, dim=-1)
              action = torch.multinomial(probabilities, num_samples=1).unsqueeze(0)
          # Take action in the environment
          new_node = env.edge_index[action[0].item()]
          next_state, reward, done, _,route = env.step(new_node)
          # Update current state
          state = next_state
          # Update list of previously taken actions
          taken_actions.append(action[0].item())
          if done :
            total_reward =0
            total_revenue = 0
            total_time = 0
            feasible = 1
            steps = len(route)-1
            for j in range(0,steps):
              reward = reward_matrix.iloc[route[j],route[j+1]]
              revenue = revenue_matrix.iloc[route[j],route[j+1]]
              time = time_matrix.iloc[route[j],route[j+1]]
              total_reward = total_reward + reward
              total_revenue = total_revenue + revenue
              total_time = total_time + time
              loads = loads_matrix.iloc[route[j],route[j+1]]
              if loads < 2:
                feasible = 0
      if total_reward > 0 and feasible == 1:
        type1=type1+1
      elif total_reward > 0 and feasible == 0:
        type2=type2+1
      elif total_reward <= 0 and feasible == 1:
        type3=type3+1
      else:
        type4=type4+1
      days = calculate_days(route, time_matrix)
      visualized_route = visualize_route(route,hub_labels)
      all_routes.append(route)
      # Output episode solution
      print("Route:",visualized_route,route,"Profit:",total_reward,"Feasible:",feasible)

Route: ['Los Angeles (CA)', 'Cheyenne (WY)', 'Eau Claire (WI)', 'Los Angeles (CA)'] [9, 41, 56, 9] Profit: -1220.8445384615384 Feasible: 1
Route: ['Los Angeles (CA)', 'Cheyenne (WY)', 'Morenci (AZ)', 'Los Angeles (CA)'] [9, 41, 3, 9] Profit: -511.9076153846149 Feasible: 1
Route: ['Los Angeles (CA)', 'Cheyenne (WY)', 'Eau Claire (WI)', 'Los Angeles (CA)'] [9, 41, 56, 9] Profit: -1220.8445384615384 Feasible: 1
Route: ['Los Angeles (CA)', 'Cheyenne (WY)', 'Eau Claire (WI)', 'Los Angeles (CA)'] [9, 41, 56, 9] Profit: -1220.8445384615384 Feasible: 1
Route: ['Los Angeles (CA)', 'Cheyenne (WY)', 'Eau Claire (WI)', 'Los Angeles (CA)'] [9, 41, 56, 9] Profit: -1220.8445384615384 Feasible: 1
Route: ['San Diego (CA)', 'Chandler (AZ)', 'Priest River (ID)', 'Carson (WA, OR)', 'San Diego (CA)'] [11, 0, 20, 40, 11] Profit: -429.03738461538455 Feasible: 1
Route: ['San Diego (CA)', 'Chandler (AZ)', 'Priest River (ID)', 'Carson (WA, OR)', 'San Diego (CA)'] [11, 0, 20, 40, 11] Profit: -429.03738461538455 

In [59]:
from collections import defaultdict
# Create a defaultdict with lists
routes_dict = defaultdict(list)

# Group routes by their starting number
for route in all_routes:
    start_number = route[0]  # Get the starting number
    routes_dict[start_number].append(route)

# Convert defaultdict to a regular dictionary (optional)
routes_dict = dict(routes_dict)

In [60]:
operative_columns = [
    "Orig", "Dest", "Miles", "Trip_type", "Avg_rate","Loads_Available", "Cargo_Type", 
    "PickUpDay", "DeliveryDay"]
def format_currency(value):
    return f"${value:.2f}"
for start_number, routes in routes_dict.items():  # Outer loop for each starting number
    operative_routes= pd.DataFrame(columns=operative_columns)
    visualized_route = visualize_route(routes[0],hub_labels)
    hub_origin = visualized_route[0]
    for route in routes:  # Inner loop for each route in the group
        operative_new_route = pd.DataFrame(np.nan, index=range(len(route)-1), columns=operative_columns)
        for lane in range(0,len(route)-1):
            operative_new_route["Orig"][lane] = visualized_route[lane]
            operative_new_route["Dest"][lane] = visualized_route[lane+1]
            distance = distance_matrix.iloc[route[lane], route[lane+1]]
            operative_new_route["Miles"][lane] = str(distance)
            operative_new_route["Cargo_Type"][lane] = "FTL"
            rate = rate_matrix.iloc[route[lane], route[lane+1]]
            avg_rev = rate * distance                          
            operative_new_route["Avg_rate"][lane] = format_currency(rate)
            availability = loads_matrix.iloc[route[lane], route[lane+1]]
            operative_new_route["Loads_Available"][lane] = str(availability)
            if availability == 0:
                operative_new_route["Trip_type"][lane] = "Deadhead"
            else:
                operative_new_route["Trip_type"][lane] = "Loaded"
        current_day = 1
        current_time = 0
        for j in range(0,len(operative_new_route)):
            if j == 0: #The first lane always is day 1
                operative_new_route["PickUpDay"][j] = current_day
                pick_ups_day = 1
            else:
                distance = float(operative_new_route["Miles"][j-1]) #Get the time consumed after the last lane
                current_time = current_time + distance
                if ((current_day - current_time/605) > 0.45) and (pick_ups_day < 2): #Check if you can Pick Up that day or move to another day
                    operative_new_route["PickUpDay"][j] = current_day
                    pick_ups_day = pick_ups_day + 1
                else: 
                    if ((np.ceil(current_time/605) - current_time/605) > 0.45) and (pick_ups_day < 2):
                        current_day = np.floor(current_time/605) + 1
                        pick_ups_day = 1
                    else:
                        current_day = np.ceil(current_time/605) + 1 
                        pick_ups_day = 1
                    #current_day = current_day + added_days
                    operative_new_route["PickUpDay"][j] = current_day
        current_time = 0
        deliveries_day = 0
        #Navigate each lane of the Route to assign Delivery Day
        for j in range(0,len(operative_new_route)):
            distance = float(operative_new_route["Miles"][j]) #Get the time consumed 
            pick_up_day = operative_new_route["PickUpDay"][j]
            current_time = current_time + distance
            if ((pick_up_day - current_time/605) > 0.45) and (deliveries_day < 2): #Check if you can deliver that day or move to another day
                current_day = pick_up_day
                operative_new_route["DeliveryDay"][j] = current_day
                deliveries_day = deliveries_day + 1
            else: 
                if ((np.ceil(current_time/605) - current_time/605) > 0.45) and (deliveries_day < 2):
                    current_day = np.floor(current_time/605) + 1
                    deliveries_day = 1
                else:
                    current_day = np.ceil(current_time/605) + 1 
                    deliveries_day = 1
                #current_day = current_day + added_days
                if current_day == pick_up_day and distance > 605:
                    current_day = current_day + 1
                operative_new_route["DeliveryDay"][j] = current_day
        operative_routes = pd.concat([operative_routes, operative_new_route], ignore_index=True)
        operative_routes.loc[len(operative_routes)] = ["end", np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
    operative_name = hub_origin+'_'+ "Operative.csv"
    operative_routes.to_csv(operative_name, header=True,index=False)